In [8]:
import os
import random
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.utils.data import Dataset
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from torch.utils.data import DataLoader
from torchinfo import summary

ModuleNotFoundError: No module named 'torchinfo'

In [2]:
if torch.cuda.is_available():       
    device = torch.device("cuda")
    print(f'There are {torch.cuda.device_count()} GPU(s) available.')
    print('Device name:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
Device name: NVIDIA GeForce RTX 3070


In [3]:
class wineDataset(Dataset):
    def __init__(self, csv_path):
        super().__init__()
        # Load data to pandas DataFrame
        df = pd.read_csv(csv_path)
        # Convert data to a NumPy array and assign to self.data
        self.data = df.to_numpy()
        
    # Implement __len__ to return the number of data samples
    def __len__(self):
        return self.data.shape[0]
    
    def __getitem__(self, idx):
        features = self.data[idx]
        # Assign last data column to label
        return features

In [4]:
train_dataloader = DataLoader(wineDataset('data/wine.csv'), batch_size=512, shuffle=True)
#create batches of data for training
real_batch = next(iter(train_dataloader))

In [19]:
real_batch[0]

tensor([ 7.9000,  0.7650,  0.0000,  2.0000,  0.0840,  9.0000, 22.0000,  0.9962,
         3.3300,  0.6800, 10.9000,  6.0000], dtype=torch.float64)

In [43]:
class Generator(nn.Module):
    def __init__(self, in_dim):
        super(Generator, self).__init__()
        self.gen = nn.Sequential(
            nn.Linear(in_dim, 32),
            nn.ReLU(),
            nn.Linear(32, 64),
            nn.ReLU(),
            nn.Linear(64, 12)
        )

    def forward(self, input):
        return self.gen(input)


In [44]:
netG = Generator(12).to(device)
print(netG)

Generator(
  (gen): Sequential(
    (0): Linear(in_features=12, out_features=32, bias=True)
    (1): ReLU()
    (2): Linear(in_features=32, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=12, bias=True)
  )
)


In [22]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.dis = nn.Sequential(
            nn.Linear(12, 24),
            nn.ReLU(),
            nn.Linear(24, 36),
            nn.ReLU(),
            nn.Linear(36, 1),
            nn.Sigmoid()
        )

    def forward(self, input):
        return self.dis(input)

In [23]:
netD = Discriminator().to(device)

In [49]:
criterion = nn.BCELoss()

real_label = 1.
fake_label = 0.

# Setup Adam optimizers for both G and D
optimizerD = optim.Adam(netD.parameters(), lr=0.0001, betas=(0.5, 0.999))
optimizerG = optim.Adam(netG.parameters(), lr=0.0001, betas=(0.5, 0.999))

In [55]:
real_label = torch.ones((1), dtype=torch.float32, device=device)
fake_label = torch.zeros((1), dtype=torch.float32, device=device)

In [63]:
# Training Loop

# Lists to keep track of progress
img_list = []
G_losses = []
D_losses = []
iters = 0
num_epochs = 100
print("Starting Training Loop...")
# For each epoch
for epoch in range(num_epochs):
    # For each batch in the dataloader
    for i, data in enumerate(train_dataloader, 0):
        ############################
        netD.zero_grad()
        # Format batch
        real_cpu = data[0].to(device).float()
        b_size = real_cpu.size(0)

        # Forward pass real batch through D
        output = netD(real_cpu).view(-1)
        # Calculate loss on all-real batch
    
        errD_real = criterion(output, real_label)
        # Calculate gradients for D in backward pass
        errD_real.backward()
        D_x = output.mean().item()

        ## Train with all-fake batch
        # Generate batch of latent vectors
        noise = torch.randn(b_size, 1, 1, device=device)
        noise = torch.transpose(noise, 1, 2)
        # Generate fake image batch with G
        fake = netG(noise.squeeze())
        
        # Classify all fake batch with D
        output = netD(fake.detach()).view(-1)
        # Calculate D's loss on the all-fake batch
        errD_fake = criterion(output, fake_label)
        # Calculate the gradients for this batch, accumulated (summed) with previous gradients
        errD_fake.backward()
        D_G_z1 = output.mean().item()
        # Compute error of D as sum over the fake and the real batches
        errD = errD_real + errD_fake
        # Update D
        optimizerD.step()

        ############################
        # (2) Update G network: maximize log(D(G(z)))
        ###########################
        netG.zero_grad()
        # Since we just updated D, perform another forward pass of all-fake batch through D
        output = netD(fake).view(-1)
        # Calculate G's loss based on this output
        errG = criterion(output, real_label)
        # Calculate gradients for G
        errG.backward()
        D_G_z2 = output.mean().item()
        # Update G
        optimizerG.step()
        print('[%d/%d][%d/%d]\tLoss_D: %.4f\tLoss_G: %.4f\tD(x): %.4f\tD(G(z)): %.4f / %.4f'
                  % (epoch, num_epochs, i, len(train_dataloader),
                     errD.item(), errG.item(), D_x, D_G_z1, D_G_z2))
        # Output training stats
        if i % 50 == 0:
            print('[%d/%d][%d/%d]\tLoss_D: %.4f\tLoss_G: %.4f\tD(x): %.4f\tD(G(z)): %.4f / %.4f'
                  % (epoch, num_epochs, i, len(train_dataloader),
                     errD.item(), errG.item(), D_x, D_G_z1, D_G_z2))

        # Save Losses for plotting later
        G_losses.append(errG.item())
        D_losses.append(errD.item())

Starting Training Loop...
[0/100][0/4]	Loss_D: 1.3849	Loss_G: 0.9552	D(x): 0.4062	D(G(z)): 0.3838 / 0.3847
[0/100][0/4]	Loss_D: 1.3849	Loss_G: 0.9552	D(x): 0.4062	D(G(z)): 0.3838 / 0.3847
[0/100][1/4]	Loss_D: 1.2399	Loss_G: 0.6953	D(x): 0.5749	D(G(z)): 0.4966 / 0.4989
[0/100][2/4]	Loss_D: 1.4942	Loss_G: 0.9391	D(x): 0.3641	D(G(z)): 0.3836 / 0.3910
[0/100][3/4]	Loss_D: 1.2684	Loss_G: 0.7524	D(x): 0.5283	D(G(z)): 0.4676 / 0.4712
[1/100][0/4]	Loss_D: 1.7004	Loss_G: 0.6802	D(x): 0.3689	D(G(z)): 0.5049 / 0.5065
[1/100][0/4]	Loss_D: 1.7004	Loss_G: 0.6802	D(x): 0.3689	D(G(z)): 0.5049 / 0.5065
[1/100][1/4]	Loss_D: 0.5427	Loss_G: 0.8911	D(x): 0.9950	D(G(z)): 0.4159 / 0.4102
[1/100][2/4]	Loss_D: 0.7393	Loss_G: 0.9315	D(x): 0.7890	D(G(z)): 0.3948 / 0.3940
[1/100][3/4]	Loss_D: 1.7706	Loss_G: 0.6753	D(x): 0.3501	D(G(z)): 0.5138 / 0.5090
[2/100][0/4]	Loss_D: 1.0659	Loss_G: 0.8796	D(x): 0.5833	D(G(z)): 0.4096 / 0.4149
[2/100][0/4]	Loss_D: 1.0659	Loss_G: 0.8796	D(x): 0.5833	D(G(z)): 0.4096 / 0.4149
[2